# Solution: Planning events for visitors

<font color="blue"><b>First task:</b></font> Let $c>0$ be the cost vector, and let $A$ be the schedule matrix, i.e., the $0$-$1$ matrix that has one column per event and one row per group, and the table entry in row $i$ and column $j$ equals $1$ iff Group $i$ is in the zoo at the time event $j$ takes place (this is the same matrix as introduced in the second task). Moreover, let $b$ be the all-ones-vector that has a $1$ for each group. We claim that the linear program

$$
\min \{ c^\top x\colon Ax \geq b,\ x\geq 0 \}
$$

is what we are looking for. First of all, note that the matrix $A$ is a consecutive ones matrix if the columns are ordered by the starting times of the corresponding events: Every group comes to the zoo at some point in time and leaves at a later one, and they can potentially attend every event that takes place in this time window. Thus, the matrix $A$ is totally unimodular, and hence also $-A$ is. By Theorem 5.8 from the script (note that $-b$ is integral), we thus obtain that

$$
P = \{x\in\mathbb{R}^n_{\geq 0}\colon -Ax \leq -b \}
$$

is an integral polytope. But $P$ is precisely the feasible region of the linear program stated earlier, hence we conclude that optimal vertex solutions are integral.

Now let $x$ be an optimal vertex solution. Note that $x$ is a $0$-$1$ vector. Indeed, non-negativity constraints are included in the linear program, and an optimal solution $x$ cannot have an entry larger than $1$: If $x$ had an entry strictly larger than $1$, one could decrease this entry to $1$ without affecting feasibility in $Ax\geq b$ (because $A$ is a $0$-$1$ matrix and $b$ is the all-ones vector). As $c>0$, this would decrease the objective, contradicting the assumption that $x$ was optimal. Hence indeed, $x$ is a $0$-$1$ vector.

We claim that choosing all events for which the corresponding entry of $x$ equals $1$ is a feasible choice in the sense that every group will be able to attend at least one of the chosen events. To see this, consider group $i$, corresponding to the $i^\text{th}$ row $A_{i.}$ of $A$. We have that

$$
1 \leq A_{i.} x = \sum_{j\in\text{Events}} A_{ij}x_j = \sum_{j\in\text{Events}} 1_{\text{group $i$ is in the zoo when event $j$ takes place}}\cdot 1_{\text{event $j$ is offered}} = |\{\text{events that are offered while group $i$ is at the zoo}\}|\enspace.
$$

Thus, the choice indicated by a vertex solution $x$ is indeed a feasible choice, as the above holds for all groups. Moreover, the total cost of the choice is given by $c^\top x$, the value of the linear program. On the other hand, it is also immediate to see that the indicator vector of any feasible choice is feasible for the linear program, whith its cost being equal to the objective function value of the indicator vector. Consequently, an optimal vertex solution of our linear program does indeed correspond to an optimal choice of events.

---

<font color="blue"><b>Second task:</b></font> We use pulp to solve the problem.

In [ ]:
costs = [3, 5, 7, 9, 6, 3, 8, 5, 7, 6, 4]
schedules = [
    [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
]

In [ ]:
import pulp

# helpers: number of events, dictionary of event names
n = len(costs)
events = range(n)
events_dict = dict([(e,f"event at {8+e}:00") for e in events])

# setting up minimization LP
lp = pulp.LpProblem("Event planning", pulp.LpMinimize)

# variables
events_var = [pulp.LpVariable(events_dict[e], lowBound = 0) for e in events]

# objective
lp += pulp.lpSum([costs[e]*events_var[e] for e in events])

# constraints
for i in range(len(schedules)):
    lp += pulp.lpSum([schedules[i][e]*events_var[e] for e in events]) >= 1
    
# solve LP
lp.solve()

# check integrality of the solution
integral = True
for e in events:
    if events_var[e].value() not in set([0,1]):
        integral = False
        break
if integral:
    print("LP Solution is 0/1.\n")
else:
    print("LP Solution is not 0/1.\n")

# read solution
print(f"An optimal choice has cost {lp.objective.value()} and " + 
      "consists of the following events:")
for e in events:
    if events_var[e].value() == 1:
        print(f" - {events_dict[e]} (cost {costs[e]})")

---

<font color="blue"><b>Bonus task:</b></font> We propose the following algorithm.

  1. Let $G$ be the set of all groups.
  2. while ($G$ is non-empty):
  
     - Choose the earliest event $e_i$ such that for some group $g_i\in G$, it is the last event that they can potentially visit.
     
     - Delete all groups that can visit event $e_i$ from $G$.
  3. Return the set of all chosen events $e_i$.

Let $E=\{e_1,\ldots,e_k\}$ be the set of events that are chosen by the above algorithm for a given instance, and let $g_1,\ldots,g_k$ be the corresponding groups in $G$ such that $e_i$ was the last event that $g_i$ could potentially attend, as stated in step 2. above.

**Proof of correctness of the algorithm.** Obviously, every group that is in $G$ initially can attend at least one of the events in $E$, because groups are only deleted from $G$ after choosing an event $e_i$ that they can attend.

To see that the choice is one of minimal cardinality, we claim that for no two groups $g_i$ and $g_j$, there is an event that they can both attend. Once we proved this claim, it is evident that we need at least $k$ events, namely at least one for each of the groups $g_1,\ldots,g_k$. As previously, we saw that the $k$ events $E=\{e_1,\ldots,e_k\}$ are also enough, we know that $E$ is an optimal choice.

To prove the claim, we show that for every $i\in[k-1]$, all events that $g_{i+1}$ can attend take place later than all events that $g_{i}$ can attend. This is true because after the $i^{th}$ iteration of the while-loop, all groups that can attend events before or including $e_i$ were deleted from $G$ (formally, this can be proved by induction). Thus, group $g_{i+1}$ can only attend events that take place after $e_i$, hence in particular after all events that $g_i$ can attend.

**Proof of efficiency.** The while-loop can be implemented by looping over all $n$ events in increasing order of time (i.e., looping over the columns of the input matrix), always checking whether the current event is one that should be chosen (which can be checked by iterating over the groups once). This implementation thus gives an $O(mn)$ running time bound.